# Exericse 1  
In this exercise you will be implementing a simple aligner. You will use to to align the following read:
'ACATACACATGTCCTGTTTTGATGTCCTATAATTAATTTTCTCTCCGTTTTTAACTTTTATCTATCTTATTAATGT'  
to the refernece sequence located in the file example_reference.fasta (contig 20)

**NOTE: example_reference.fasta  and example_reference.fasta.fai need to be copied from public files to the project in order to have them available.** 

## Seed phase
- Implement a simple hash-based aligner in Python
    - A dict can be used to create the index
    - Create an index for each kmer in a sequence
    - Create this index for the example fasta file 
        (in the data directory, used in Week 5)  
          
          
- To map a read, find locations for each kmer in the read
    - Mind the offset from the beginning of the read
    - Find the region with most kmers mapping to it
    
- collections.Counter() may be useful (pass a list to Counter(), and it will return the number of times each element occurs in the list)

In [7]:
!pip install pysam

In [8]:
# loading libraries
import pysam
import sys
from collections import Counter
from itertools import chain

Reading necessary data: reference sequence chr 20, read sequence, kmer size

In [9]:
fasta = pysam.FastaFile("/sbgenomics/project-files/example_human_reference.fasta").fetch('20')
read = 'ACATACACATGTCCTGTTTTGATGTCCTATAATTAATTTTCTCTCCGTTTTTAACTTTTATCTATCTTATTAATGT'
k = 10

In [11]:
#fasta

**STEP 1: Create index for provided fasta file**

In [14]:
#STEP 1.1 create all k-mers from the fasta

kmers = []
for i in range(len(fasta) - k):
    kmers.append(fasta[i:i+k])

print(len(kmers))
print(len(set(kmers)))

9991
9759


In [15]:
# STEP 1.2 save the position for the k-mer

kmers = {}
for i in range(len(fasta) - k):
    kmer = fasta[i:i+k]
    if kmer in kmers:
        kmers[kmer].append(i)
    else:
        kmers[kmer] = [i]


In [18]:
# kmers

In [16]:
len(kmers)

9759

In [21]:
# function to create index out of reference
def create_index(fasta, k):
    kmers = {}
    for i, x in enumerate(fasta[:-k]):
        kmer = fasta[i:i+k]
        if kmer not in kmers:
            kmers[kmer] = []
        kmers[kmer].append(i)
    return kmers


In [22]:
len(create_index(fasta, k))

9759

**STEP 2: Get number of unique k-mers and number kmers with of colisions**

In [23]:
index = create_index(fasta, k)
print("Number of unique k-mers:", len(index))
print("Number of k-mers with collisions:", len({k:v for k,v in index.items() if len(v)>1}))

Number of unique k-mers: 9759
Number of k-mers with collisions: 213


**STEP 3: Create seed function which will return dict with reference positions as keys and with number of supporting kmers as values**

In [25]:
# treba read da rastavimo na kmere i za svaki od kmera vidimo gde se sve pojavljuje u genomu 
def seed_read(index, k, read):
    read_seeds = []
    for i in range(len(read) - k + 1): # za svaku poziciju u readu
        read_kmer = read[i:i+k]
        positions = index[read_kmer]
        for position in positions:
            read_seeds.append(position - i)

    return Counter(read_seeds)

In [26]:
seed_read(index,k, read)

Counter({5793: 67, 792: 2})

In [34]:
# more pythonic way to write this code
def seed_read1(index, k, read):
    return Counter(chain.from_iterable([[y-i for y in index.get(read[i:i+k], [])] for i in range(len(read) - k +1)]))

In [35]:
seed_read1(index,k, read)

Counter({5793: 67, 792: 2})

**STEP 4: Possible improvements? Filter out all kmers that have more than n mapping positions**

In [27]:
def seed_read2_with_improvements(index, k, read, n=1):
    read_seeds = []
    for i in range(len(read) - k + 1): # za svaku poziciju u readu
        read_kmer = read[i:i+k]
        positions = index[read_kmer]
        if len(positions)<=n:
            for position in positions:
                read_seeds.append(position - i)

    return Counter(read_seeds)

In [28]:
seed_read2_with_improvements(index, k, read)

Counter({5793: 65})